In [0]:
# Installs gee  package
import subprocess

try:
    
    import ee
except ImportError:
    print('gee package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'ee'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
except:
    
# Authenticates and initializes Earth Engine
    import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [0]:
# Set start and end date
import datetime
startTime = datetime.datetime(2013, 2, 3)
endTime = datetime.datetime(2019, 5, 25)

# Create image collection
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT').filterDate(startTime, endTime)
precipitation = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').filterDate(startTime, endTime)
probav = ee.ImageCollection('VITO/PROBAV/C1/S1_TOC_100M').filterDate(startTime, endTime)


In [0]:
#Retrieve information over image collection
point = {'type':'Point', 'coordinates':[-99.545934,20.572173]};

info_image = image.getRegion(point,500).getInfo()

In [25]:
# Reshape Landsat 8
import pandas as pd
import matplotlib.dates as mdates
from IPython.display import Image
from pylab import *
from numpy import fft

import matplotlib.pyplot as plt


header = info_image[0]
data = array(info_image[1:])

iTime = header.index('time')
time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(int))]

# List of used image bands
band_list = ['B5','B4']

iBands = [header.index(b) for b in band_list]
yData = data[0:,iBands].astype(np.float)

# Calculate NDVI
red = yData[:,0]
nir = yData[:,1]
ndvi = (nir - red) / (nir + red)

# Reshape NDVI array into Pandas Dataframe
df = pd.DataFrame(data=ndvi*-1, index=time, columns=['ndvi'])
df_red=pd.DataFrame(data=red, index=time, columns=['red'])
l8_time_series = df.dropna()

# Resampling
monthly_landsat = l8_time_series.resample('M', how='mean')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: FutureWarning:

how in .resample() is deprecated
the new syntax is .resample(...).mean()



In [0]:
monthly = pd.DataFrame({'landsat': monthly_landsat['ndvi']})

In [24]:
monthly

,landsat
2013-04-30,0.078764
2013-05-31,0.085864
2013-06-30,0.153815
2013-07-31,0.221417
2013-08-31,0.129672
...,...
2019-01-31,0.070517
2019-02-28,0.088748
2019-03-31,0.100357
2019-04-30,0.093038
